# Exploration du jeu de données 

*Pour récupérer le Dataset, on a automatisé ça en récupérant le token de l'API Kaggle*  

*Pour récupérer le Dataset :*  
- Aller sur [Kaggle](https://www.kaggle.com/).  
- Se connecter à son compte ou s'inscrire.  
- Cliquer en haut à droite sur l'icône de son profil.  
- Aller dans la rubrique API de l'onglet settings puis "create new token".    

*Le token de l'API Kaggle est maintenant téléchargé.*  

In [ ]:
import os
import json

kaggle_path = os.path.expanduser("~/.kaggle")
kaggle_json_path = os.path.expanduser("~/Downloads/kaggle.json")

with open(kaggle_json_path, "r") as f:
    kaggle_token = json.load(f)

with open(os.path.join(kaggle_path, "kaggle.json"), "w") as f:
    json.dump(kaggle_token, f)

os.chmod(os.path.join(kaggle_path, "kaggle.json"), 0o600)

In [ ]:
!kaggle datasets download -d mikoajkolman/pokemon-images-first-generation17000-files --unzip -p "/Users/gabriel/pokemon_nn/images_pokemon"

Dataset URL: https://www.kaggle.com/datasets/mikoajkolman/pokemon-images-first-generation17000-files
License(s): GPL-2.0
100%|██████████████████████████████████████| 2.81G/2.81G [04:31<00:00, 10.2MB/s]
100%|██████████████████████████████████████| 2.81G/2.81G [04:31<00:00, 11.1MB/s]


*Le jeu de données est déjà très propre nous n'avons pas à le clean*